<a href="https://colab.research.google.com/github/kunakl07/Backend-of-Active-Learning/blob/pre_spec_al/Case_one/Preprocessing/preprocess_case_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#![Flowchart](https://drive.google.com/file/d/1r0IVntROxR92vBDzbHn1aRgH0HFYLlu5/view?usp=sharing)



![Here is the flowchart of the steps that I hve taken in my notebook](https://drive.google.com/uc?export=view&id=1DnCGt0QxE1W7JDyiS35ayDv3GQGCSM8Q)

# Contents

- [Preprocessing](#Preprocessing-Stage)  
  - [Importing packages](#Importing-the-necessary-packages)  
  - [Standardize](#Standardize-the-tsv-files)  
  - [Extract positive and negative calls from audio using tsv's](#Extract)  
  - [Plot Spectrograms](#Spec-generate)
  - [Same steps for test data](#test-data)  
- [Training Stage](#Training-Stage)  
   - [Basic CNN model](#Basic-CNN-model)
   - [VGG16-model](#VGG-16-model)

#**Preprocessing Stage**



###Downloading the PodcastR2,PodcastR3 and Podcast_Test files and extracting them.
####Since these files contains the calls of SRKW, only podcastR2 and podcastR3 have been downloaded and used

In [ ]:
!apt-get -qq install awscli
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab07052019_PodCastRound2.tar.gz ./ 
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab09272017_PodCastRound3.tar.gz ./
!aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/test/OrcasoundLab09272017_Test.tar.gz ./
!tar -xzf OrcasoundLab09272017_PodCastRound3.tar.gz
!tar -xzf OrcasoundLab07052019_PodCastRound2.tar.gz
!tar -xzf OrcasoundLab09272017_Test.tar.gz
!pip -q install ketos==2.0.0b4
!pip -q install pysoundfile
!pip install pydub


Selecting previously unselected package sgml-base.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../00-sgml-base_1.29_all.deb ...
Unpacking sgml-base (1.29) ...
Selecting previously unselected package python3-yaml.
Preparing to unpack .../01-python3-yaml_3.12-1build2_amd64.deb ...
Unpacking python3-yaml (3.12-1build2) ...
Selecting previously unselected package python3-six.
Preparing to unpack .../02-python3-six_1.11.0-2_all.deb ...
Unpacking python3-six (1.11.0-2) ...
Selecting previously unselected package python3-dateutil.
Preparing to unpack .../03-python3-dateutil_2.6.1-1_all.deb ...
Unpacking python3-dateutil (2.6.1-1) ...
Selecting previously unselected package xml-core.
Preparing to unpack .../04-xml-core_0.18_all.deb ...
Unpacking xml-core (0.18) ...
Selecting previously unselected package docutils-common.
Preparing to unpack .../05-docutils-common_0.14+dfsg-3_all.deb ...
Unpacking docutils-common (0.14+dfsg-3) ...
Selecting prev

##Preprocessing on positive train dataset

###Importing the necessary packages 

In [ ]:
import pandas as pd
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.audio.spectrogram import MagSpectrogram
from ketos.data_handling.parsing import load_audio_representation
import numpy as np
from os import listdir
from os.path import isfile, join
from scipy import signal
import soundfile as sf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pydub import AudioSegment


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [ ]:
#Generate mean and annot-train
def duration_mean(filename):
    annot_train = pd.read_csv(filename, sep='\t')
    mean=annot_train['duration_s'].mean()
    return annot_train,mean
  

In [ ]:
#Function to add the end time
def add_end(filename):
    filename["end"]=filename["start"]+filename["duration_s"]


In [ ]:
#Function to find extract filename and start time
def fname_stime(filename):
    file_name=filename.iloc[:,0].values
    start_time=filename.iloc[:,1].values
    return file_name,start_time


In [ ]:
#Function to extract audio from the .wav files to generate complete positive and negative calls
def extract_audio(label,filename,path,position):
    file_name=filename.iloc[:,0].values
    start_time=filename.iloc[:,position].values
    i=0
    o=0
    for x in file_name:
  
        AUDIO_FILE=x
        sound = AudioSegment.from_file(AUDIO_FILE)
        p=start_time[i]
        p=p*1000
        print(p)
        i=i+1
        o=p+2000
        call=sound[p:o]
        call.export(path+label+ "calls{0}.wav".format(i),format="wav")


In [ ]:
#Plotting the spectrogram
def plot_spectrogram(base,plot,calls):
    basePath = base
    plotPath = join(basePath,plot)
    folderpath = join(basePath, calls)
    onlyfiles = [f for f in listdir(folderpath) if isfile(join(join(folderpath, f)))]
    
    for idx, file in enumerate(onlyfiles):
        data, samplerate = sf.read(join(folderpath, file))
        f, t, spec = signal.spectrogram(data, samplerate)
        filename = file.split(sep=".")[0]
    
        fig, ax = plt.subplots(1, 1)
        ax.specgram(data, Fs=samplerate, NFFT=1024)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        scale_y = 1000
        ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    
        plt.savefig(join(plotPath, 
                 filename + ".png"))
        plt.close(fig)

####The tsv files contains the parameters like start_time,duration_s,etc, but since these are not in the format Ketos accepts,we need to perform some changes in labels and therefore these files have been uploaded from the local machine


In [ ]:
annot_train2,mean2=duration_mean('/content/podcast2.tsv')
annot_train3,mean3 = duration_mean('/content/podcast3.tsv')
annot_test,mean_test = duration_mean('/content/v10_test.tsv')

print(mean2)
annot_train2.head()


2.1110548004254963


,wav_filename,start,duration_s,location,date,data_source,data_source_id,label
0,1562337136_0004.wav,49.765625,2.450000,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
1,1562337136_0004.wav,41.046007,1.658854,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
2,1562337136_0004.wav,37.345486,1.743924,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
3,1562337136_0004.wav,42.917535,2.594618,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
4,1562337136_0004.wav,45.980035,2.041667,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs


####Here is how the .tsv files and their labels look like when they are in format that could be accepted by ketos

In [ ]:
add_end(annot_train2)
add_end(annot_train3)
add_end(annot_test)
annot_train2.head()

,wav_filename,start,duration_s,location,date,data_source,data_source_id,label,end
0,1562337136_0004.wav,49.765625,2.450000,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,52.215625
1,1562337136_0004.wav,41.046007,1.658854,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,42.704861
2,1562337136_0004.wav,37.345486,1.743924,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,39.089410
3,1562337136_0004.wav,42.917535,2.594618,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,45.512153
4,1562337136_0004.wav,45.980035,2.041667,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,48.021701


###Standardizing the tsv files

In [ ]:
map_to_ketos_annot_std ={'wav_filename': 'filename'} 
std_annot_train2 = sl.standardize(table=annot_train2, signal_labels=["SRKWs"], mapper=map_to_ketos_annot_std, trim_table=True)
std_annot_train3 = sl.standardize(table=annot_train3, signal_labels=["SRKWs"], mapper=map_to_ketos_annot_std, trim_table=True)

std_annot_test = sl.standardize(table=annot_test, signal_labels=["SRKWs"], mapper=map_to_ketos_annot_std, trim_table=True)


###Here we could see how each these tsv files look like after standardizing

In [ ]:
std_annot_train2.head()


start  label        end
filename            annot_id                             
1562337136_0004.wav 0         49.765625      1  52.215625
                    1         41.046007      1  42.704861
                    2         37.345486      1  39.089410
                    3         42.917535      1  45.512153
                    4         45.980035      1  48.021701

In [ ]:
std_annot_train3.head()

start  label        end
filename                        annot_id                             
OS_9_27_2017_08_14_00__0002.wav 0          6.110451      1   7.856295
OS_9_27_2017_08_14_00__0004.wav 0         12.717882      1  15.167882
                                1         29.825347      1  31.637326
                                2         43.504514      1  45.103819
                                3         48.404514      1  50.344097

In [ ]:
std_annot_test.head()

start  label        end
filename                        annot_id                             
OS_9_27_2017_08_14_00__0001.wav 0         11.643564      1  14.093564
                                1         15.594059      1  17.759901
                                2         53.900000      1  56.350000
                                3         59.781486      1  61.250000
OS_9_27_2017_08_19_00__0002.wav 0          6.592882      1   7.826389

###Saving these standardized tsv files

In [ ]:
std_annot_train2.to_csv('standardized_train2.tsv', mode='a', sep='\t',header=False)
std_annot_train3.to_csv('standardized_train3.tsv', mode='a', sep='\t',header=False)
std_annot_test.to_csv('standardized_test.tsv', mode='a', sep='\t',header=False)

In [ ]:
annot_id2 = pd.read_csv('/content/standardized_train2.tsv', sep='\t')
annot_id3 = pd.read_csv('/content/standardized_train3.tsv', sep='\t')
annot_idtest = pd.read_csv('/content/standardized_test.tsv', sep='\t')


####Extracting the .wav file names and start time from these .tsv files which would be used by Pydub to extract small segemets of sounds(one containing the calls and the other not)

In [ ]:
#Function to find extract filename and start time
def fname_stime(filename):
    file_name=filename.iloc[:,0].values
    start_time=filename.iloc[:,1].values
    return file_name,start_time


In [ ]:
std_annot_train2.head()

start  label        end
filename            annot_id                             
1562337136_0004.wav 0         49.765625      1  52.215625
                    1         41.046007      1  42.704861
                    2         37.345486      1  39.089410
                    3         42.917535      1  45.512153
                    4         45.980035      1  48.021701

In [ ]:
filename,start_time=fname_stime(std_annot_train2)
print(filename[0])
print(start_time[0])
annot_train2.head()

49.765625
1


,wav_filename,start,duration_s,location,date,data_source,data_source_id,label,end
0,1562337136_0004.wav,49.765625,2.450000,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,52.215625
1,1562337136_0004.wav,41.046007,1.658854,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,42.704861
2,1562337136_0004.wav,37.345486,1.743924,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,39.089410
3,1562337136_0004.wav,42.917535,2.594618,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,45.512153
4,1562337136_0004.wav,45.980035,2.041667,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs,48.021701


####We could verify the start time and the file duration matches to the same column as above

####We would change to the directory from where we want to extract calls using pydub

In [ ]:
!pwd

/content


In [ ]:
%cd /content/Round2_OS_07_05/wav

/content/Round2_OS_07_05/wav


In [ ]:
!mkdir pod_calls

In [ ]:
extract_audio('round2_calls',annot_train2,"/content/Round2_OS_07_05/wav/pod_calls/",1)

####Extracting the start time plus two second sound which we know by taking the mean of the duration

In [ ]:
%cd /content/Round3_OS_09_27_2017/wav

/content/Round3_OS_09_27_2017/wav


####Similarly we would extract the calls for podcast3

In [ ]:
extract_audio('round3_calls',annot_train3,"/content/Round2_OS_07_05/wav/pod_calls/",1)

####Verify the calls that we extracted are of annotated three itself

In [ ]:
annot_train3.head()

,wav_filename,start,duration_s,location,date,data_source,data_source_id,label,end
0,OS_9_27_2017_08_14_00__0002.wav,6.110451,1.745843,orcasound_lab,2017-09-27,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,7.856295
1,OS_9_27_2017_08_14_00__0004.wav,12.717882,2.450000,orcasound_lab,2017-09-27,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,15.167882
2,OS_9_27_2017_08_14_00__0004.wav,29.825347,1.811979,orcasound_lab,2017-09-27,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,31.637326
3,OS_9_27_2017_08_14_00__0004.wav,43.504514,1.599306,orcasound_lab,2017-09-27,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,45.103819
4,OS_9_27_2017_08_14_00__0004.wav,48.404514,1.939583,orcasound_lab,2017-09-27,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,50.344097


In [ ]:
%cd /content/Round2_OS_07_05

/content/Round2_OS_07_05


###In the Round2 folder we are going to create two folders train and test respectively and in each of the folders we are going to create calls and nocalls folders respectively 

In [ ]:
!mkdir train
!mkdir test
%cd train
!mkdir calls
!mkdir nocalls
%cd /content/Round2_OS_07_05/test

/content/Round2_OS_07_05/train
/content/Round2_OS_07_05/test


In [ ]:
'''filename='/content/Round2_OS_07_05/wav/pod_calls/MMMcalls102.wav'
siz = wavread(filename,'size') 
siz = [samples channels]
siz(1)/Fs #%should give you the length in second
'''

"filename='/content/Round2_OS_07_05/wav/pod_calls/MMMcalls102.wav'\nsiz = wavread(filename,'size') \nsiz = [samples channels]\nsiz(1)/Fs #%should give you the length in second\n"

In [ ]:
!pwd
!mkdir calls
!mkdir nocalls

/content/Round2_OS_07_05/test


###Now we would plot the graphs i.e the spectrograms without x and y labels into calls folder

In [ ]:
plot_spectrogram('/content/Round2_OS_07_05/','train/calls','wav/pod_calls')

##Generation of spectrograms from the background sounds 

####Since we have generated the positive calls, its time to generate the negative ones.

###The above table shows the time area for the podcast-2 and podcast-3 which does not contain the calls timeframe
###The table displays the start time and the end time that does not contain the calls

In [ ]:
positives_train2 = sl.select(annotations=std_annot_train2, length=2.0)
file_durations_train2 = sl.file_duration_table('/content/Round2_OS_07_05/wav')
negatives_train2=sl.create_rndm_backgr_selections(annotations=std_annot_train2, files=file_durations_train2, length=2.0, num=len(positives_train2), trim_table=True)
negatives_train2.head()

start        end  label
filename            sel_id                             
1562337136_0004.wav 0        0.316021   2.316021      0
                    1       11.787732  13.787732      0
                    2       13.420318  15.420318      0
                    3       21.158559  23.158559      0
                    4       23.745995  25.745995      0

####Extracting the area by looking at time where there are no occurences of calls in tsv file 

In [ ]:
positives_train3 = sl.select(annotations=std_annot_train3, length=2.0)
file_durations_train33 = sl.file_duration_table('/content/Round3_OS_09_27_2017/wav')
negatives_train33=sl.create_rndm_backgr_selections(annotations=std_annot_train3, files=file_durations_train33, length=2.0, num=len(positives_train3), trim_table=True)
negatives_train33.head()

start        end  label
filename                        sel_id                             
OS_9_27_2017_08_03_00__0002.wav 0       31.586918  33.586918      0
OS_9_27_2017_08_03_00__0003.wav 0        9.968256  11.968256      0
                                1       15.586086  17.586086      0
                                2       17.013745  19.013745      0
                                3       50.955001  52.955001      0

###Then the steps for generating audio is same as we did above for the positive calls

In [ ]:
!pwd

/content/Round2_OS_07_05/test


In [ ]:
%cd '/content/Round2_OS_07_05/wav'

/content/Round2_OS_07_05/wav


In [ ]:
!mkdir neg_pod_calls

In [ ]:
negatives_train2.to_csv('negative2.tsv', mode='a', sep='\t',header=False)
negatives_train33.to_csv('negative3.tsv', mode='a', sep='\t',header=False)

In [ ]:
negatives_train2save=pd.read_csv('/content/Round2_OS_07_05/test/negative2.tsv',sep='\t')
negatives_train33save=pd.read_csv('/content/Round2_OS_07_05/test/negative3.tsv',sep='\t')

In [ ]:
negatives_train2save.head()

,1562337136_0004.wav,0,4.965906564760641,6.965906564760641,0.1
0,1562337136_0004.wav,1,8.060243,10.060243,0
1,1562337136_0004.wav,2,13.916050,15.916050,0
2,1562337136_0004.wav,3,19.756478,21.756478,0
3,1562337136_0004.wav,4,23.582382,25.582382,0
4,1562337136_0004.wav,5,28.850221,30.850221,0


In [ ]:
extract_audio('r2negcalls_calls',negatives_train2save,"/content/Round2_OS_07_05/wav/neg_pod_calls/",2)

In [ ]:
%cd '/content/Round3_OS_09_27_2017/wav'

/content/Round3_OS_09_27_2017/wav


In [ ]:
extract_audio('r3neg_calls',negatives_train33save,"/content/Round2_OS_07_05/wav/neg_pod_calls/",2)

In [ ]:
negatives_train33save.head()

,OS_9_27_2017_08_03_00__0002.wav,0,31.586917768911015,33.586917768911015,0.1
0,OS_9_27_2017_08_03_00__0003.wav,0,9.968256,11.968256,0
1,OS_9_27_2017_08_03_00__0003.wav,1,15.586086,17.586086,0
2,OS_9_27_2017_08_03_00__0003.wav,2,17.013745,19.013745,0
3,OS_9_27_2017_08_03_00__0003.wav,3,50.955001,52.955001,0
4,OS_9_27_2017_08_09_00__0000.wav,0,0.236795,2.236795,0


In [ ]:
!pwd

/content/Round3_OS_09_27_2017/wav


In [ ]:
import os
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
from pydub import AudioSegment


# Generate mean and annot-train
def duration_mean(filename):
    annot_train = pd.read_csv(filename, sep='\t')
    mean = annot_train['duration_s'].mean()
    return annot_train, mean


# Function to find extract filename and start time
def fname_stime(filename):
    file_name = filename.iloc[:, 0].values
    start_time = filename.iloc[:, 1].values
    return file_name, start_time


# Function to extract audio from the .wav files to generate
# complete positive and negative calls
def extract_audio(label, filename, path, position, file_location):
    file_name = filename.iloc[:, 0].values
    start_time = filename.iloc[:, position].values
    i = 0
    o = 0
    os.chdir(file_location)

    for x in file_name:
        AUDIO_FILE = x
        sound = AudioSegment.from_file(AUDIO_FILE)
        p = start_time[i]
        p = p * 1000
        print(p)
        i = i + 1
        o = p + 3000
        call = sound[p:o]
        call.export(path + label + "MMMcalls{0}.wav".format(i), format="wav")


#Plotting the spectrogram
def plot_spectrogram(base,plot,calls):
    basePath = base
    plotPath = join(basePath,plot)
    folderpath = join(basePath, calls)
    onlyfiles = [f for f in listdir(folderpath) if isfile(join(join(folderpath, f)))]
    for idx, file in enumerate(onlyfiles):
        data, samplerate = sf.read(join(folderpath, file))
        f, t, spec = signal.spectrogram(data, samplerate)
        filename = file.split(sep=".")[0]
    
        fig, ax = plt.subplots(1, 1)
        ax.specgram(data, Fs=samplerate, NFFT=1024)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)


        scale_y = 1000
        ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
        os.chdir(plotPath)

        plt.savefig(join(plotPath, 
                 filename + ".png"))
        plt.close(fig)


# Enter the path of the standardized tsv's
annot_train2, mean2 = duration_mean('/content/podcast2.tsv')
annot_train3, mean3 = duration_mean('/content/podcast3.tsv')
annot_test, mean_test = duration_mean('/content/v10_test.tsv')


# Extract the audio of the calls
extract_audio(
    "round2_calls", annot_train2,
    "/content/pod_calls/", 1,
    "/content/Round2_OS_07_05/wav/")

extract_audio(
    'round3_calls', annot_train3,
    "/content/pod_calls/", 1,
    "/content/Round3_OS_09_27_2017/wav/")


#Standardizing the tsv files
map_to_ketos_annot_std ={'wav_filename': 'filename'} 
std_annot_train2 = sl.standardize(table=annot_train2, signal_labels=["SRKWs"], 
                       mapper=map_to_ketos_annot_std, trim_table=True)
std_annot_train3 = sl.standardize(table=annot_train3, signal_labels=["SRKWs"], 
                       mapper=map_to_ketos_annot_std, trim_table=True)
std_annot_test = sl.standardize(table=annot_test, signal_labels=["SRKWs"], 
                       mapper=map_to_ketos_annot_std, trim_table=True)



# Since we also want the negative audio, we would generate negative sound 
# by extracting it from the background sound what are not present in  the call
# duration. For that first we create a tsv that generates time-interval from the
# tsv files that are not within the start-time and end-time
positives_train2 = sl.select(annotations=std_annot_train2, length=3.0)
file_durations_train2 = sl.file_duration_table('/content/Round2_OS_07_05/wav')
negatives_train2=sl.create_rndm_backgr_selections(
                    annotations=std_annot_train2, 
                    files=file_durations_train2, length=3.0, 
                    num=len(positives_train2), 
                    trim_table=True)


# Same steps for podcast3 tsv file
positives_train3 = sl.select(annotations=std_annot_train3, length=3.0)
file_durations_train33 = sl.file_duration_table('/content/Round3_OS_09_27_2017/wav')
negatives_train33=sl.create_rndm_backgr_selections(
                    annotations=std_annot_train3, 
                    files=file_durations_train33, 
                    length=3.0, num=len(positives_train3), 
                    trim_table=True)


#Saving these tsv files for future use
negatives_train2.to_csv('negative2.tsv', mode='a', sep='\t',header=False)
negatives_train33.to_csv('negative3.tsv', mode='a', sep='\t',header=False)
negatives_train2save=pd.read_csv('/content/Round2_OS_07_05/test/negative2.tsv',sep='\t')
negatives_train33save=pd.read_csv('/content/Round2_OS_07_05/test/negative3.tsv',sep='\t')



extract_audio(
    'round2_calls', negatives_train2save,
    "/content/pod_calls/", 2,
    "/content/Round2_OS_07_05/wav/")


extract_audio(
    'round3_calls', negatives_train33save,
    "/content/pod_calls/", 2,
    "/content/Round3_OS_09_27_2017/wav/")


plot_spectrogram('/content/Round2_OS_07_05/','train/calls','wav/pod_calls')
plot_spectrogram('/content/Round2_OS_07_05/','train/nocalls','wav/neg_pod_calls')


In [ ]:
plot_spectrogram('/content/Round2_OS_07_05/','train/nocalls','wav/neg_pod_calls')


###Now we have saved the images(spectrograms) of the negtive calls 

##We would perform similar steps with test data as well where in :

*   Extract area responsible for the call from the Orcasound_test tsv file
*   Standardize that tsv file
*   Generate 2 second calls that contains the audio data
*   Generate background sound from tsv of 2 seconds 
*   Generate spectrograms from these sounds 



---







In [ ]:
annot_test.head()

,wav_filename,start,duration_s,location,date,data_source,data_source_id,label,end
0,OS_9_27_2017_08_14_00__0001.wav,11.643564,2.450000,orcasound_lab,9/27/2017,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,14.093564
1,OS_9_27_2017_08_14_00__0001.wav,15.594059,2.165842,orcasound_lab,9/27/2017,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,17.759901
2,OS_9_27_2017_08_14_00__0001.wav,53.900000,2.450000,orcasound_lab,9/27/2017,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,56.350000
3,OS_9_27_2017_08_14_00__0001.wav,59.781486,1.468514,orcasound_lab,9/27/2017,Orcasound_PodCast_Round3,OS_9_27_2017_08_14,SRKWs,61.250000
4,OS_9_27_2017_08_19_00__0002.wav,6.592882,1.233507,orcasound_lab,9/27/2017,Orcasound_PodCast_Round3,OS_9_27_2017_08_19,SRKWs,7.826389


In [ ]:
extract_audio('test_calls',annot_test,"/content/Round2_OS_07_05/wav/pod_calls_test/",1)

In [ ]:
%cd /content/Round3_OS_09_27_2017/wav/OrcasoundLab09272017_Test/wav

/content/Round3_OS_09_27_2017/wav/OrcasoundLab09272017_Test/wav


In [ ]:
plot_spectrogram('/content/Round2_OS_07_05/','test/calls','wav/pod_calls_test')
#plot_spectrogram('/content/Round2_OS_07_05/','train/nocalls','wav/neg_pod_calls')

In [ ]:
positives_test = sl.select(annotations=std_annot_test, length=2.0)
file_durations_test = sl.file_duration_table('/content/Round3_OS_09_27_2017/wav/OrcasoundLab09272017_Test/wav')
negatives_test=sl.create_rndm_backgr_selections(annotations=std_annot_test, files=file_durations_test, length=2.0, num=len(positives_test), trim_table=True)
negatives_test.to_csv('negg.tsv', mode='a', sep='\t',header=False)

In [ ]:
%cd '/content/Round3_OS_09_27_2017/wav'

/content/Round3_OS_09_27_2017/wav


In [ ]:
neg=pd.read_csv('/content/negg.tsv',sep='\t')
extract_audio('r3neg_calls_test',neg,"/content/Round2_OS_07_05/wav/neg_pod_calls_test/",2)

In [ ]:
neg.head()

,OS_9_27_2017_08_14_00__0001.wav,0,1.9261489230579651,3.926148923057965,0.1
0,OS_9_27_2017_08_14_00__0001.wav,1,32.092360,34.092360,0
1,OS_9_27_2017_08_19_00__0002.wav,0,8.783855,10.783855,0
2,OS_9_27_2017_08_19_00__0002.wav,1,10.833785,12.833785,0
3,OS_9_27_2017_08_19_00__0002.wav,2,17.738476,19.738476,0
4,OS_9_27_2017_08_19_00__0002.wav,3,33.540716,35.540716,0


In [ ]:
plot_spectrogram('/content/Round2_OS_07_05/','test/nocalls','wav/neg_pod_calls_test')

In [ ]:
!zip -r /content/train_nopreprocess.zip /content/Round2_OS_07_05/train

In [ ]:
!zip -r /content/negtrainsave.zip /content/Round2_OS_07_05/train/nocalls

In [ ]:
!zip -r /content/no_preprocess_test.zip /content/Round2_OS_07_05/test